In [1]:
# Refactoring NASEM model
import nasem_dairy as nd
import pandas as pd
import math
from pprint import pprint


# use package dir to always use /data folder regardless of where this is executed from
import importlib_resources

path_to_package_data = importlib_resources.files("nasem_dairy.data")


# Read_csv to load required data into env
user_diet, animal_input, equation_selection = nd.read_csv_input(path_to_package_data.joinpath("./input.csv"))

# Load feed library
feed_library = pd.read_csv(path_to_package_data.joinpath("NASEM_feed_library.csv"))

In [2]:
def dev_NASEM_model(user_diet, animal_input, equation_selection, feed_library_df, coeff_dict):
    """Execute NASEM functions. 
    """
    ########################################
    # Step 1: Read User Input
    ########################################
    # prevent mutable changes outside of expected scope (especially for Shiny):
    user_diet = user_diet.copy()
    animal_input = animal_input.copy()

    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    list_of_feeds = user_diet['Feedstuff'].tolist()
    # feed_data = fl_get_rows(list_of_feeds, path_to_db)
    feed_data = nd.fl_get_feed_rows(list_of_feeds, feed_library_df)


    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    # list_of_feeds = user_diet['Feedstuff'].tolist()
    # feed_data = nd.fl_get_rows(list_of_feeds, '../../src/nasem_dairy/data/diet_database.db')

    feed_data = feed_data.reset_index(names='Feedstuff')

    diet_info_initial = pd.DataFrame({'Feedstuff': user_diet['Feedstuff']})
    diet_info_initial = diet_info_initial.merge(feed_data, how='left', on='Feedstuff')

    # Add Fd_DMInp to the diet_info dataframe
    Fd_DMInp = user_diet.set_index('Feedstuff')['kg_user'] / user_diet['kg_user'].sum()

    diet_info_initial.insert(1, 'Fd_DMInp', 
        Fd_DMInp.reindex(
            diet_info_initial['Feedstuff']).values)

    diet_info_initial['Fd_DMIn'] = diet_info_initial['Fd_DMInp'] * \
        animal_input['DMI']      # Should this be done after DMI equations?

    # Add Fd_DNDF48 column, need to add to the database
    # diet_info_initial['Fd_DNDF48'] = 0

    # Calculate additional physiology values
    animal_input['An_PrePartDay'] = animal_input['An_GestDay'] - animal_input['An_GestLength']
    animal_input['An_PrePartWk'] = animal_input['An_PrePartDay'] / 7

    del (list_of_feeds, Fd_DMInp)

    # Check equation_selection to make sure they are integers.
    # This is especially important for Shiny, which may return strings
    # It's important they are correct for if statements below.
    equation_selection_in = equation_selection.copy()
    equation_selection = {}

    for key, value in equation_selection_in.items():
        try:
            num_value = int(value)
            equation_selection[key] = num_value
        except ValueError:
            print(f"Unable to convert '{value}' to an integer for key '{key}'")
    
    del(equation_selection_in)

    # if animal_input['An_StatePhys'] != 'Lactating Cow':
    #     animal_input['Trg_MilkProd'] = None

############################################################################################
# Start of refactored NASEM 
############################################################################################
            
    # Create infusion_input
    if equation_selection['use_infusions'] == 0:
        # If no infusions then all values are set to 0
        no_infusion_input = {'Inf_Acet_g': 0,
                             'Inf_ADF_g': 0,
                             'Inf_Arg_g': 0,
                             'Inf_Ash_g': 0,
                             'Inf_Butr_g': 0,
                             'Inf_CP_g': 0,
                             'Inf_CPARum_CP': 0,
                             'Inf_CPBRum_CP': 0,
                             'Inf_CPCRum_CP': 0,
                             'Inf_dcFA': 0,
                             'Inf_dcRUP': 0,
                             'Inf_DM_g': 0,
                             'Inf_EE_g': 0,
                             'Inf_FA_g': 0,
                             'Inf_Glc_g': 0,
                             'Inf_His_g': 0,
                             'Inf_Ile_g': 0,
                             'Inf_KdCPB': 0,
                             'Inf_Leu_g': 0,
                             'Inf_Lys_g': 0,
                             'Inf_Met_g': 0,
                             'Inf_NDF_g': 0,
                             'Inf_NPNCP_g': 0,
                             'Inf_Phe_g': 0,
                             'Inf_Prop_g': 0,
                             'Inf_St_g': 0,
                             'Inf_Thr_g': 0,
                             'Inf_Trp_g': 0,
                             'Inf_ttdcSt': 0,
                             'Inf_Val_g': 0,
                             'Inf_VFA_g': 0,
                             'Inf_Location': 0
                             }
    elif equation_selection['use_infusions'] == 1:
        infusion_input = nd.read_infusion_input('./infusion_input.csv')
    else:
        raise ValueError(
            f"Invalid use_infusions: {equation_selection['use_infusions']} was entered. Must be 0 or 1")

########################################
# Step 1.5: Input Based Equations
########################################
    K_FeCPend_ClfLiq = nd.calculate_K_FeCPend_ClfLiq(animal_input['An_StatePhys'],
                                                     equation_selection['NonMilkCP_ClfLiq'])

########################################
# Step 2: DMI Equations
########################################
    # # Need to precalculate Dt_NDF for DMI predicitons, this will be based on the user entered DMI (animal_input['DMI])
    Dt_NDF = (diet_info_initial['Fd_NDF'] *
              diet_info_initial['Fd_DMInp']).sum()

    if equation_selection['DMIn_eqn'] == 0:
        # print('Using user input DMI')
        pass

    # Predict DMI for lactating cow
    elif equation_selection['DMIn_eqn'] == 8:
        # print("using DMIn_eqn: 8")
        animal_input['DMI'] = nd.calculate_Dt_DMIn_Lact1(
            animal_input['An_Parity_rl'],
            animal_input['Trg_MilkProd'],
            animal_input['An_BW'],
            animal_input['An_BCS'],
            animal_input['An_LactDay'],
            animal_input['Trg_MilkFatp'],
            animal_input['Trg_MilkTPp'],
            animal_input['Trg_MilkLacp'])

    # Predict DMI for heifers
    elif equation_selection['DMIn_eqn'] in [2, 3, 4, 5, 6, 7, 12, 13, 14, 15, 16, 17]:
        animal_input['DMI'] = nd.heifer_growth(
            equation_selection['DMIn_eqn'],
            # diet_info.loc['Diet', 'Fd_NDF'],
            Dt_NDF,
            animal_input['An_BW'],
            animal_input['An_BW_mature'],
            animal_input['An_PrePartWk'],
            nd.coeff_dict)

    elif equation_selection['DMIn_eqn'] in [10, 11]:
        animal_input['DMI'] = nd.dry_cow_equations(
            equation_selection['DMIn_eqn'],
            animal_input['An_BW'],
            animal_input['An_PrePartWk'],
            animal_input['An_GestDay'],
            animal_input['An_GestLength'],
            Dt_NDF,
            nd.coeff_dict)

    else:
        # It needs to catch all possible solutions, otherwise it's possible that it stays unchanged without warning
        print("DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: " + str(
            equation_selection['DMIn_eqn']))

    # Calculated again as part of diet_data, value may change depending on DMIn_eqn selections
    del (Dt_NDF)

########################################
# Step 3: Feed Based Calculations
########################################
    # Calculate An_DMIn_BW with the final DMI value; required for calculating diet_data_initial
    An_DMIn_BW = nd.calculate_An_DMIn_BW(animal_input['An_BW'],
                                         animal_input['DMI'])
    Fe_rOMend = nd.calculate_Fe_rOMend(animal_input['DMI'],
                                       nd.coeff_dict)

    diet_info = nd.calculate_diet_info(animal_input['DMI'],
                                       animal_input['An_StatePhys'],
                                       equation_selection['Use_DNDF_IV'],
                                       diet_info=diet_info_initial,
                                       coeff_dict=nd.coeff_dict)
    # All equations in the f dataframe go into calculate_diet_info()
    # This includes micronutrient calculations which are no longer handled by seperate functions

    diet_data_initial = nd.calculate_diet_data_initial(diet_info,
                                                       animal_input['DMI'],
                                                       animal_input['An_BW'],
                                                       animal_input['An_StatePhys'],
                                                       An_DMIn_BW,
                                                       Fe_rOMend,
                                                       nd.coeff_dict)
    # diet_data contains everything starting with "Dt_"

########################################
# Step 4: Infusion Calculations
########################################
    if equation_selection['use_infusions'] == 0:
        infusion_data = nd.calculate_infusion_data(no_infusion_input,
                                                   animal_input['DMI'],
                                                   nd.coeff_dict)
    elif equation_selection['use_infusions'] == 1:
        infusion_data = nd.calculate_infusion_data(infusion_input,
                                                   animal_input['DMI'],
                                                   nd.coeff_dict)
    else:
        raise ValueError(
            f"Invalid use_infusions: {equation_selection['use_infusions']} was entered. Must be 0 or 1")

########################################
# Step 5: Animal Level Calculations
########################################
# Combine Diet and Infusion nutrient supplies
    An_data_initial = nd.calculate_An_data_initial(animal_input,
                                                   diet_data_initial,
                                                   infusion_data,
                                                   nd.coeff_dict)

########################################
# Step 6: Rumen Digestion Calculations
########################################
    # Rumen Digestability Coefficients
    Rum_dcNDF = nd.calculate_Rum_dcNDF(animal_input['DMI'],
                                       diet_data_initial['Dt_NDFIn'],
                                       diet_data_initial['Dt_StIn'],
                                       diet_data_initial['Dt_CPIn'],
                                       diet_data_initial['Dt_ADFIn'],
                                       diet_data_initial['Dt_ForWet'])

    Rum_dcSt = nd.calculate_Rum_dcSt(animal_input['DMI'],
                                     diet_data_initial['Dt_ForNDF'],
                                     diet_data_initial['Dt_StIn'],
                                     diet_data_initial['Dt_ForWet'])

    # Rumen Digestable Intakes
    Rum_DigNDFIn = nd.calculate_Rum_DigNDFIn(Rum_dcNDF,
                                             diet_data_initial['Dt_NDFIn'])

    Rum_DigStIn = nd.calculate_Rum_DigStIn(Rum_dcSt,
                                           diet_data_initial['Dt_StIn'])

########################################
# Step 7: Microbial Protein Calculations
########################################
    if equation_selection['MiN_eqn'] == 1:
        RDPIn_MiNmax = nd.calculate_RDPIn_MiNmax(animal_input['DMI'],
                                                 An_data_initial['An_RDP'],
                                                 An_data_initial['An_RDPIn'])
        MiN_Vm = nd.calculate_MiN_Vm(RDPIn_MiNmax,
                                     nd.coeff_dict)
        Du_MiN_g = nd.calculate_Du_MiN_NRC2021_g(MiN_Vm,
                                                 Rum_DigNDFIn,
                                                 Rum_DigStIn,
                                                 An_data_initial['An_RDPIn_g'],
                                                 nd.coeff_dict)
    elif equation_selection['MiN_eqn'] == 2:
        Du_MiN_g = nd.calculate_Du_MiN_VTln_g(diet_data_initial['Dt_rOMIn'],
                                              diet_data_initial['Dt_ForNDFIn'],
                                              An_data_initial['An_RDPIn'],
                                              Rum_DigStIn,
                                              Rum_DigNDFIn,
                                              nd.coeff_dict)
    elif equation_selection['MiN_eqn'] == 3:
        Du_MiN_g = nd.calculate_Du_MiN_VTnln_g(An_data_initial['An_RDPIn'],
                                               Rum_DigNDFIn,
                                               Rum_DigStIn)
    else:
        raise ValueError(
            f"Invalid MiN_eqn: {equation_selection['MiN_eqn']} was entered. Must choose 1, 2 or 3.")

    Du_MiCP_g = nd.calculate_Du_MiCP_g(Du_MiN_g)
    Du_MiTP_g = nd.calculate_Du_MiTP_g(Du_MiCP_g, nd.coeff_dict)
    Du_MiCP = nd.calculate_Du_MiCP(Du_MiCP_g)
    Du_idMiCP_g = nd.calculate_Du_idMiCP_g(Du_MiCP_g,
                                           nd.coeff_dict)
    Du_idMiCP = nd.calculate_Du_idMiCP(Du_idMiCP_g)

########################################
# Step 7.1: Fe_CP Calculation /  Finish Dt_ and An_ calculations
########################################
# Required to finish Dt_ and An_ calculations
    Fe_RUP = nd.calculate_Fe_RUP(An_data_initial['An_RUPIn'],
                                 infusion_data['InfSI_TPIn'],
                                 An_data_initial['An_idRUPIn'])
    Fe_RumMiCP = nd.calculate_Fe_RumMiCP(Du_MiCP,
                                         Du_idMiCP)
    Fe_CPend_g = nd.calculate_Fe_CPend_g(animal_input['An_StatePhys'],
                                         An_data_initial['An_DMIn'],
                                         An_data_initial['An_NDF'],
                                         animal_input['DMI'],
                                         diet_data_initial['Dt_DMIn_ClfLiq'],
                                         K_FeCPend_ClfLiq)
    Fe_CPend = nd.calculate_Fe_CPend(Fe_CPend_g)
    Fe_CP = nd.calculate_Fe_CP(animal_input['An_StatePhys'],
                               diet_data_initial['Dt_CPIn_ClfLiq'],
                               diet_data_initial['Dt_dcCP_ClfDry'],
                               An_data_initial['An_CPIn'],
                               Fe_RUP,
                               Fe_RumMiCP,
                               Fe_CPend,
                               infusion_data['InfSI_NPNCPIn'],
                               nd.coeff_dict)

########################################
# Step 7.2: Complete diet_data and An_data
########################################
    diet_data = nd.calculate_diet_data_complete(diet_data_initial,
                                                animal_input['An_StatePhys'],
                                                Fe_CP,
                                                equation_selection,
                                                nd.coeff_dict)
    An_data = nd.calculate_An_data_complete(An_data_initial,
                                            diet_data,
                                            animal_input['An_StatePhys'],
                                            Fe_CP,
                                            infusion_data,
                                            equation_selection,
                                            nd.coeff_dict)

########################################
# Step 8: Amino Acid Calculations
########################################
    AA_list = ['Arg', 'His', 'Ile', 'Leu',
               'Lys', 'Met', 'Phe', 'Thr', 'Trp', 'Val']
    AA_values = pd.DataFrame(index=AA_list)
    # Dataframe for storing all individual amino acid values

    AA_values['Du_AAMic'] = nd.calculate_Du_AAMic(Du_MiTP_g,
                                                  AA_list,
                                                  nd.coeff_dict)
    AA_values['Du_IdAAMic'] = nd.calculate_Du_IdAAMic(AA_values['Du_AAMic'],
                                                      nd.coeff_dict)
    AA_values['Abs_AA_g'] = nd.calculate_Abs_AA_g(diet_data,
                                                  AA_values['Du_IdAAMic'],
                                                  AA_list)
    AA_values['mPrtmx_AA'] = nd.calculate_mPrtmx_AA(AA_list,
                                                    nd.coeff_dict)
    f_mPrt_max = nd.calculate_f_mPrt_max(animal_input['An_305RHA_MlkTP'],
                                         nd.coeff_dict)
    AA_values['mPrtmx_AA2'] = nd.calculate_mPrtmx_AA2(AA_values['mPrtmx_AA'],
                                                      f_mPrt_max)
    AA_values['AA_mPrtmx'] = nd.calculate_AA_mPrtmx(AA_list,
                                                    nd.coeff_dict)
    AA_values['mPrt_AA_01'] = nd.calculate_mPrt_AA_01(AA_values['AA_mPrtmx'],
                                                      AA_list,
                                                      nd.coeff_dict)
    AA_values['mPrt_k_AA'] = nd.calculate_mPrt_k_AA(AA_values['mPrtmx_AA2'],
                                                    AA_values['mPrt_AA_01'],
                                                    AA_values['AA_mPrtmx'])

########################################
# Step 9: Other Calculations
########################################
    Uter_Wtpart = nd.calculate_Uter_Wtpart(
        animal_input['Fet_BWbrth'], nd.coeff_dict)
    Uter_Wt = nd.calculate_Uter_Wt(animal_input['An_Parity_rl'],
                                   animal_input['An_AgeDay'],
                                   animal_input['An_LactDay'],
                                   animal_input['An_GestDay'],
                                   animal_input['An_GestLength'],
                                   Uter_Wtpart,
                                   nd.coeff_dict)
    GrUter_Wtpart = nd.calculate_GrUter_Wtpart(
        animal_input['Fet_BWbrth'], nd.coeff_dict)
    GrUter_Wt = nd.calculate_GrUter_Wt(animal_input['An_GestDay'],
                                       animal_input['An_GestLength'],
                                       Uter_Wt,
                                       GrUter_Wtpart,
                                       nd.coeff_dict)
    Uter_BWgain = nd.calculate_Uter_BWgain(animal_input['An_LactDay'],
                                           animal_input['An_GestDay'],
                                           animal_input['An_GestLength'],
                                           Uter_Wt,
                                           nd.coeff_dict)
    GrUter_BWgain = nd.calculate_GrUter_BWgain(animal_input['An_LactDay'],
                                               animal_input['An_GestDay'],
                                               animal_input['An_GestLength'],
                                               GrUter_Wt,
                                               Uter_BWgain,
                                               nd.coeff_dict)

    return animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data

# ### RUN MODEL ###
# animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data = run_NASEM()

# Run model - different options

In [3]:
animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data = \
    dev_NASEM_model(user_diet, animal_input, equation_selection, feed_library_df = feed_library, coeff_dict = nd.coeff_dict)

In [4]:
diet_info

,Feedstuff,Fd_DMInp,Fd_Libr,UID,Fd_Index,Fd_Category,Fd_Type,Fd_DM,Fd_Conc,Fd_Locked,...,Fd_IdThrRUPIn,Fd_IdTrpRUPIn,Fd_IdValRUPIn,Fd_DigSt,Fd_DigStIn_Base,Fd_DigrOMt,Fd_DigrOMtIn,Fd_idRUPIn,TT_dcFdFA,Fd_DigFAIn
0,Alfalfa meal,0.334821,NRC 2020,NRC16F1,1,Plant Protein,Forage,90.749,0,1,...,16.585398,6.845611,21.079355,1.66530,0.136724,21.724818,1.783641,0.396934,73.0,0.096374
1,Canola meal,0.274554,NRC 2020,NRC16F28,28,Plant Protein,Concentrate,89.128,100,1,...,28.868286,8.708639,34.456598,1.42870,0.096185,17.015330,1.145528,0.611287,73.0,0.123553
2,"Corn silage, typical",0.223214,NRC 2020,NRC16F48,48,Grain Crop Forage,Forage,35.361,0,1,...,3.268773,0.696060,4.460466,29.25163,1.601070,11.935807,0.653299,0.089708,73.0,0.093977
3,"Corn grain HM, coarse grind",0.167411,NRC 2020,NRC16F1072,256,Energy Source,Concentrate,72.267,100,1,...,4.422491,0.908995,5.827574,63.83520,2.620485,5.798527,0.238034,0.114339,73.0,0.107012
